In [1]:
import pandas as pd
rock_samples = pd.read_csv('data/rocksamples.csv')
rock_samples.head()

ID   Mission    Type   Subtype  Weight (g)  Pristine (%)
0  10001  Apollo11    Soil  Unsieved       125.8         88.36
1  10002  Apollo11    Soil  Unsieved      5629.0         93.73
2  10003  Apollo11  Basalt  Ilmenite       213.0         65.56
3  10004  Apollo11    Core  Unsieved        44.8         71.76
4  10005  Apollo11    Core  Unsieved        53.4         40.31

In [2]:
rock_samples.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2229 entries, 0 to 2228
Data columns (total 6 columns):
 #   Column        Non-Null Count  Dtype  
---  ------        --------------  -----  
 0   ID            2229 non-null   int64  
 1   Mission       2229 non-null   object 
 2   Type          2229 non-null   object 
 3   Subtype       2226 non-null   object 
 4   Weight (g)    2229 non-null   float64
 5   Pristine (%)  2229 non-null   float64
dtypes: float64(2), int64(1), object(3)
memory usage: 104.6+ KB


In [3]:
rock_samples['Weight (g)'] = rock_samples['Weight (g)'].apply(lambda x : x * 0.001)
rock_samples.rename(columns={'Weight (g)':'Weight (kg)'}, inplace=True)
rock_samples.head()

ID   Mission    Type   Subtype  Weight (kg)  Pristine (%)
0  10001  Apollo11    Soil  Unsieved       0.1258         88.36
1  10002  Apollo11    Soil  Unsieved       5.6290         93.73
2  10003  Apollo11  Basalt  Ilmenite       0.2130         65.56
3  10004  Apollo11    Core  Unsieved       0.0448         71.76
4  10005  Apollo11    Core  Unsieved       0.0534         40.31

In [4]:
missions = pd.DataFrame()
missions['Mission'] = rock_samples['Mission'].unique()
missions.head()

Mission
0  Apollo11
1  Apollo12
2  Apollo14
3  Apollo15
4  Apollo16

In [5]:
missions.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 6 entries, 0 to 5
Data columns (total 1 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   Mission  6 non-null      object
dtypes: object(1)
memory usage: 176.0+ bytes


In [6]:
sample_total_weight = rock_samples.groupby('Mission')['Weight (kg)'].sum()
missions = pd.merge(missions, sample_total_weight, on='Mission')
missions.rename(columns={'Weight (kg)':'Sample weight (kg)'}, inplace=True)
missions

Mission  Sample weight (kg)
0  Apollo11            21.55424
1  Apollo12            34.34238
2  Apollo14            41.83363
3  Apollo15            75.39910
4  Apollo16            92.46262
5  Apollo17           109.44402

In [7]:
sample_total_weight = rock_samples.groupby('Mission')['Weight (kg)'].sum()
sample_total_weight

Mission
Apollo11     21.55424
Apollo12     34.34238
Apollo14     41.83363
Apollo15     75.39910
Apollo16     92.46262
Apollo17    109.44402
Name: Weight (kg), dtype: float64

In [8]:
missions['Weight diff'] = missions['Sample weight (kg)'].diff()
missions

Mission  Sample weight (kg)  Weight diff
0  Apollo11            21.55424          NaN
1  Apollo12            34.34238     12.78814
2  Apollo14            41.83363      7.49125
3  Apollo15            75.39910     33.56547
4  Apollo16            92.46262     17.06352
5  Apollo17           109.44402     16.98140

In [9]:
missions['Weight diff'] = missions['Weight diff'].fillna(value=0)
missions

Mission  Sample weight (kg)  Weight diff
0  Apollo11            21.55424      0.00000
1  Apollo12            34.34238     12.78814
2  Apollo14            41.83363      7.49125
3  Apollo15            75.39910     33.56547
4  Apollo16            92.46262     17.06352
5  Apollo17           109.44402     16.98140

In [10]:
missions['Lunar module (LM)'] = {'Eagle (LM-5)', 'Intrepid (LM-6)', 'Antares (LM-8)', 'Falcon (LM-10)', 'Orion (LM-11)', 'Challenger (LM-12)'}
missions['LM mass (kg)'] = {15103, 15235, 15264, 16430, 16445, 16456}
missions['LM mass diff'] = missions['LM mass (kg)'].diff()
missions['LM mass diff'] = missions['LM mass diff'].fillna(value=0)

missions['Command module (CM)'] = {'Columbia (CSM-107)', 'Yankee Clipper (CM-108)', 'Kitty Hawk (CM-110)', 'Endeavor (CM-112)', 'Casper (CM-113)', 'America (CM-114)'}
missions['CM mass (kg)'] = {5560, 5609, 5758, 5875, 5840, 5960}
missions['CM mass diff'] = missions['CM mass (kg)'].diff()
missions['CM mass diff'] = missions['CM mass diff'].fillna(value=0)

missions

Mission  Sample weight (kg)  Weight diff   Lunar module (LM)  \
0  Apollo11            21.55424      0.00000        Eagle (LM-5)   
1  Apollo12            34.34238     12.78814     Intrepid (LM-6)   
2  Apollo14            41.83363      7.49125      Falcon (LM-10)   
3  Apollo15            75.39910     33.56547  Challenger (LM-12)   
4  Apollo16            92.46262     17.06352       Orion (LM-11)   
5  Apollo17           109.44402     16.98140      Antares (LM-8)   

   LM mass (kg)  LM mass diff      Command module (CM)  CM mass (kg)  \
0         15264           0.0       Columbia (CSM-107)          5960   
1         15235         -29.0          Casper (CM-113)          5609   
2         16456        1221.0  Yankee Clipper (CM-108)          5840   
3         16430         -26.0        Endeavor (CM-112)          5875   
4         16445          15.0         America (CM-114)          5560   
5         15103       -1342.0      Kitty Hawk (CM-110)          5758   

   CM mass diff  
0           0.0  
1        -351.0  
2         231.0  
3          35.0  
4        -315.0  
5         198.0

In [11]:
missions['Total weight (kg)'] = missions['LM mass (kg)'] + missions['CM mass (kg)']
missions['Total weight diff'] = missions['LM mass diff'] + missions['CM mass diff']
missions

Mission  Sample weight (kg)  Weight diff   Lunar module (LM)  \
0  Apollo11            21.55424      0.00000        Eagle (LM-5)   
1  Apollo12            34.34238     12.78814     Intrepid (LM-6)   
2  Apollo14            41.83363      7.49125      Falcon (LM-10)   
3  Apollo15            75.39910     33.56547  Challenger (LM-12)   
4  Apollo16            92.46262     17.06352       Orion (LM-11)   
5  Apollo17           109.44402     16.98140      Antares (LM-8)   

   LM mass (kg)  LM mass diff      Command module (CM)  CM mass (kg)  \
0         15264           0.0       Columbia (CSM-107)          5960   
1         15235         -29.0          Casper (CM-113)          5609   
2         16456        1221.0  Yankee Clipper (CM-108)          5840   
3         16430         -26.0        Endeavor (CM-112)          5875   
4         16445          15.0         America (CM-114)          5560   
5         15103       -1342.0      Kitty Hawk (CM-110)          5758   

   CM mass diff  Total weight (kg)  Total weight diff  
0           0.0              21224                0.0  
1        -351.0              20844             -380.0  
2         231.0              22296             1452.0  
3          35.0              22305                9.0  
4        -315.0              22005             -300.0  
5         198.0              20861            -1144.0

In [12]:
# Sample-to-weight ratio
saturnVPayload = 43500
missions['Crewed area : Payload'] = missions['Total weight (kg)'] / saturnVPayload
missions['Sample : Crewed area'] = missions['Sample weight (kg)'] / missions['Total weight (kg)']
missions['Sample : Payload'] = missions['Sample weight (kg)'] / saturnVPayload
missions

Mission  Sample weight (kg)  Weight diff   Lunar module (LM)  \
0  Apollo11            21.55424      0.00000        Eagle (LM-5)   
1  Apollo12            34.34238     12.78814     Intrepid (LM-6)   
2  Apollo14            41.83363      7.49125      Falcon (LM-10)   
3  Apollo15            75.39910     33.56547  Challenger (LM-12)   
4  Apollo16            92.46262     17.06352       Orion (LM-11)   
5  Apollo17           109.44402     16.98140      Antares (LM-8)   

   LM mass (kg)  LM mass diff      Command module (CM)  CM mass (kg)  \
0         15264           0.0       Columbia (CSM-107)          5960   
1         15235         -29.0          Casper (CM-113)          5609   
2         16456        1221.0  Yankee Clipper (CM-108)          5840   
3         16430         -26.0        Endeavor (CM-112)          5875   
4         16445          15.0         America (CM-114)          5560   
5         15103       -1342.0      Kitty Hawk (CM-110)          5758   

   CM mass diff  Total weight (kg)  Total weight diff  Crewed area : Payload  \
0           0.0              21224                0.0               0.487908   
1        -351.0              20844             -380.0               0.479172   
2         231.0              22296             1452.0               0.512552   
3          35.0              22305                9.0               0.512759   
4        -315.0              22005             -300.0               0.505862   
5         198.0              20861            -1144.0               0.479563   

   Sample : Crewed area  Sample : Payload  
0              0.001016          0.000495  
1              0.001648          0.000789  
2              0.001876          0.000962  
3              0.003380          0.001733  
4              0.004202          0.002126  
5              0.005246          0.002516

In [13]:
crewedArea_payload_ratio = missions['Crewed area : Payload'].mean()
sample_crewedArea_ratio = missions['Sample : Crewed area'].mean()
sample_payload_ratio = missions['Sample : Payload'].mean()
print(crewedArea_payload_ratio)
print(sample_crewedArea_ratio)
print(sample_payload_ratio)

0.49630268199233724
0.0028946732226251396
0.0014369195019157093


In [14]:
artemis_crewedArea = 26520
artemis_mission = pd.DataFrame({'Mission':['artemis1','artemis1b','artemis2'],
                                 'Total weight (kg)':[artemis_crewedArea,artemis_crewedArea,artemis_crewedArea],
                                 'Payload (kg)':[26988, 37965, 42955]})
artemis_mission

Mission  Total weight (kg)  Payload (kg)
0   artemis1              26520         26988
1  artemis1b              26520         37965
2   artemis2              26520         42955

In [15]:
artemis_mission['Sample weight from total (kg)'] = artemis_mission['Total weight (kg)'] * sample_crewedArea_ratio
artemis_mission['Sample weight from payload (kg)'] = artemis_mission['Payload (kg)'] * sample_payload_ratio
artemis_mission

Mission  Total weight (kg)  Payload (kg)  Sample weight from total (kg)  \
0   artemis1              26520         26988                      76.766734   
1  artemis1b              26520         37965                      76.766734   
2   artemis2              26520         42955                      76.766734   

   Sample weight from payload (kg)  
0                        38.779584  
1                        54.552649  
2                        61.722877

In [16]:
artemis_mission['Estimated sample weight (kg)'] = (artemis_mission['Sample weight from payload (kg)'] + artemis_mission['Sample weight from total (kg)'])/2
artemis_mission

Mission  Total weight (kg)  Payload (kg)  Sample weight from total (kg)  \
0   artemis1              26520         26988                      76.766734   
1  artemis1b              26520         37965                      76.766734   
2   artemis2              26520         42955                      76.766734   

   Sample weight from payload (kg)  Estimated sample weight (kg)  
0                        38.779584                     57.773159  
1                        54.552649                     65.659691  
2                        61.722877                     69.244806

In [17]:
rock_samples['Remaining (kg)'] = rock_samples['Weight (kg)'] * (rock_samples['Pristine (%)'] * .01)
rock_samples.head()

ID   Mission    Type   Subtype  Weight (kg)  Pristine (%)  \
0  10001  Apollo11    Soil  Unsieved       0.1258         88.36   
1  10002  Apollo11    Soil  Unsieved       5.6290         93.73   
2  10003  Apollo11  Basalt  Ilmenite       0.2130         65.56   
3  10004  Apollo11    Core  Unsieved       0.0448         71.76   
4  10005  Apollo11    Core  Unsieved       0.0534         40.31   

   Remaining (kg)  
0        0.111157  
1        5.276062  
2        0.139643  
3        0.032148  
4        0.021526

In [18]:
rock_samples.describe()

ID  Weight (kg)  Pristine (%)  Remaining (kg)
count   2229.000000  2229.000000   2229.000000     2229.000000
mean   52058.432032     0.168253     84.512764        0.138103
std    26207.651471     0.637286     22.057299        0.525954
min    10001.000000     0.000000      0.000000        0.000000
25%    15437.000000     0.003000     80.010000        0.002432
50%    65527.000000     0.010200     92.300000        0.008530
75%    72142.000000     0.093490     98.140000        0.078240
max    79537.000000    11.729000    180.000000       11.169527

In [19]:
low_samples = rock_samples.loc[(rock_samples['Weight (kg)'] >= .16) & (rock_samples['Pristine (%)'] <= 50)]
low_samples.head()

ID   Mission     Type   Subtype  Weight (kg)  Pristine (%)  \
11  10017  Apollo11   Basalt  Ilmenite        0.973         43.71   
14  10020  Apollo11   Basalt  Ilmenite        0.425         27.88   
15  10021  Apollo11  Breccia  Regolith        0.250         30.21   
29  10045  Apollo11   Basalt   Olivine        0.185         12.13   
37  10057  Apollo11   Basalt  Ilmenite        0.919         35.15   

    Remaining (kg)  
11        0.425298  
14        0.118490  
15        0.075525  
29        0.022441  
37        0.323028

In [20]:
low_samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 27 entries, 11 to 2183
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              27 non-null     int64  
 1   Mission         27 non-null     object 
 2   Type            27 non-null     object 
 3   Subtype         27 non-null     object 
 4   Weight (kg)     27 non-null     float64
 5   Pristine (%)    27 non-null     float64
 6   Remaining (kg)  27 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.7+ KB


In [21]:
low_samples.Type.unique()

array(['Basalt', 'Breccia', 'Soil', 'Core'], dtype=object)

In [22]:
rock_samples.Type.unique()

array(['Soil', 'Basalt', 'Core', 'Breccia', 'Special', 'Crustal'],
      dtype=object)

In [23]:
low_samples.groupby('Type')['Weight (kg)'].count()

Type
Basalt     14
Breccia     8
Core        1
Soil        4
Name: Weight (kg), dtype: int64

In [24]:
needed_samples = low_samples[low_samples['Type'].isin(['Basalt', 'Breccia'])]
needed_samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 22 entries, 11 to 2183
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              22 non-null     int64  
 1   Mission         22 non-null     object 
 2   Type            22 non-null     object 
 3   Subtype         22 non-null     object 
 4   Weight (kg)     22 non-null     float64
 5   Pristine (%)    22 non-null     float64
 6   Remaining (kg)  22 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 1.4+ KB


In [25]:
needed_samples.groupby('Type')['Weight (kg)'].sum()

Type
Basalt     17.4234
Breccia    10.1185
Name: Weight (kg), dtype: float64

In [26]:
rock_samples.groupby('Type')['Weight (kg)'].sum()

Type
Basalt      93.14077
Breccia    168.88075
Core        19.93587
Crustal      4.74469
Soil        87.58981
Special      0.74410
Name: Weight (kg), dtype: float64

In [27]:
needed_samples = needed_samples.append(rock_samples.loc[rock_samples['Type'] == 'Crustal'])
needed_samples.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 68 entries, 11 to 2189
Data columns (total 7 columns):
 #   Column          Non-Null Count  Dtype  
---  ------          --------------  -----  
 0   ID              68 non-null     int64  
 1   Mission         68 non-null     object 
 2   Type            68 non-null     object 
 3   Subtype         68 non-null     object 
 4   Weight (kg)     68 non-null     float64
 5   Pristine (%)    68 non-null     float64
 6   Remaining (kg)  68 non-null     float64
dtypes: float64(3), int64(1), object(3)
memory usage: 4.2+ KB


In [28]:
needed_samples_overview = pd.DataFrame()
needed_samples_overview['Type'] = needed_samples.Type.unique()
needed_samples_overview

Type
0   Basalt
1  Breccia
2  Crustal

In [29]:
needed_sample_weights = needed_samples.groupby('Type')['Weight (kg)'].sum().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, needed_sample_weights, on='Type')
needed_samples_overview.rename(columns={'Weight (kg)':'Total weight (kg)'}, inplace=True)
needed_samples_overview

Type  Total weight (kg)
0   Basalt           17.42340
1  Breccia           10.11850
2  Crustal            4.74469

In [30]:
needed_sample_ave_weights = needed_samples.groupby('Type')['Weight (kg)'].mean().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, needed_sample_ave_weights, on='Type')
needed_samples_overview.rename(columns={'Weight (kg)':'Average weight (kg)'}, inplace=True)
needed_samples_overview

Type  Total weight (kg)  Average weight (kg)
0   Basalt           17.42340             1.244529
1  Breccia           10.11850             1.264812
2  Crustal            4.74469             0.103145

In [31]:
total_rock_count = rock_samples.groupby('Type')['ID'].count().reset_index()
needed_samples_overview = pd.merge(needed_samples_overview, total_rock_count, on='Type')
needed_samples_overview.rename(columns={'ID':'Number of samples'}, inplace=True)
total_rocks = needed_samples_overview['Number of samples'].sum()
needed_samples_overview['Percentage of rocks'] = needed_samples_overview['Number of samples'] / total_rocks
needed_samples_overview

Type  Total weight (kg)  Average weight (kg)  Number of samples  \
0   Basalt           17.42340             1.244529                351   
1  Breccia           10.11850             1.264812                959   
2  Crustal            4.74469             0.103145                 46   

   Percentage of rocks  
0             0.258850  
1             0.707227  
2             0.033923

In [32]:
artemis_ave_weight = artemis_mission['Estimated sample weight (kg)'].mean()
artemis_ave_weight

64.22588520079607

In [33]:
needed_samples_overview['Weight to collect'] = needed_samples_overview['Percentage of rocks'] * artemis_ave_weight
needed_samples_overview['Rocks to collect'] = needed_samples_overview['Weight to collect'] / needed_samples_overview['Average weight (kg)']
needed_samples_overview

Type  Total weight (kg)  Average weight (kg)  Number of samples  \
0   Basalt           17.42340             1.244529                351   
1  Breccia           10.11850             1.264812                959   
2  Crustal            4.74469             0.103145                 46   

   Percentage of rocks  Weight to collect  Rocks to collect  
0             0.258850          16.624842         13.358345  
1             0.707227          45.422289         35.912271  
2             0.033923           2.178754         21.123128